In [5]:
# The process will be this:
# 1. Read all dimensions into pandas dfs
# 2. Per record of the fact load, search in the dimension dataframes for the keys to apply
# 3. write the values of these keys into a new dataframe that will be the final df load
# 4. load adjusted dataframe to MSSQL

In [6]:
import pandas as pd

from sqlalchemy import create_engine

from dischargerecord import FactDischarge

In [7]:
# Connect to database
engine = create_engine('mssql://LAPTOP-TH3PDN0I/Group_8_DB?driver=ODBC+Driver+17+for+SQL+Server')

In [13]:
# Load Dimensions
dim_date_df = pd.read_sql_table('DimDate', con=engine)
dim_location_df = pd.read_sql_table('DimLocation', con=engine)
dim_demographics_df = pd.read_sql_table('DimDemographics', con=engine)
dim_payment_df = pd.read_sql_table('DimPayment', con=engine)
dim_clinic_class_df = pd.read_sql_table('DimClinicClass', con=engine)
dim_apr_class_df = pd.read_sql_table('DimAPRClassification', con=engine)
dim_admission_df = pd.read_sql_table('DimAdmission', con=engine)
dim_provider_df = pd.read_sql_table('DimProvider', con=engine)

In [45]:
# Load facts from csv
fact_discharge_df = pd.read_csv('./sample.csv', header=0)
print(fact_discharge_df)

   Health Service Area Hospital County  Operating Certificate Number  \
0           Western NY     Cattaraugus                        401001   
1           Western NY     Cattaraugus                        401001   
2           Western NY     Cattaraugus                        401001   
3           Western NY     Cattaraugus                        401001   
4           Western NY      Chautauqua                        602001   
5           Western NY            Erie                       1401014   
6           Western NY            Erie                       1401014   
7           Western NY            Erie                       1401014   
8           Western NY            Erie                       1401014   
9           Western NY            Erie                       1401014   
10          Western NY            Erie                       1401014   
11          Western NY            Erie                       1401014   
12          Western NY            Erie                       140

In [43]:
# Iterate fact dataframe

for fact in fact_discharge_df.itertuples():
    fact_record = {}

    fact_date_key = dim_date_df.loc[dim_date_df['DischargeYear'] == fact[14]]['DateKey'][0]
    fact_provider_key = dim_provider_df.loc[
        dim_provider_df['AttendingLicenseNo'] == fact[30] &
        dim_provider_df['OperatingLicenseNo'] == str(fact[31]) &
        dim_provider_df['OtherLicenseNo'] == fact [32]
    ]['ProviderKey'][0]
    
    fact_record = {
        'ProviderKey': fact_provider_key,
        'DateKey': fact_date_key
    }
    
    print(fact_record)
    

TypeError: cannot compare a dtyped [object] array with a scalar of type [bool]

In [44]:
print(dim_provider_df)

    ProviderKey AttendingLicenseNo OperatingLicenseNo OtherLicenseNo  \
0             1             275408               None           None   
1             2             267162               None           None   
2             3             271326               None           None   
3             4             243034               None           None   
4             5             147477               None           None   
5             6             269986               None           None   
6             7             270855               None           None   
7             8             247297               None           None   
8             9           40003545               None           None   
9            10             253732               None           None   
10           11             254778               None           None   
11           12             268543               None           None   
12           13             271780               None           